# Querying Data from `FTA Bus Award StoryMap`

Currently FTA uses ArcGIS storymap to display their bus award data. Is is possible to extract this data using their REST server.
Extracting bus award data from a server will be the most reliable way to see the data.

In [1]:
import geopandas as gpd

# got geojson from ArcGIS Rest Server 
# https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query
# https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query

fy24 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

fy23 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=" 


fy_24_data = gpd.read_file(fy24)
fy_23_data = gpd.read_file(fy23)

display(
    fy_24_data.shape,
    type(fy_24_data),
    fy_24_data.columns,
    fy_23_data.shape,
    type(fy_23_data),
    fy_23_data.columns,
)


(117, 22)

geopandas.geodataframe.GeoDataFrame

Index(['Application_Number_', 'Agency_Name', 'Low_No_BBF',
       'Project_Description', 'Number_of_Buses_', 'Funding_Amount',
       'Project_Title', 'Project_Type', 'Display_Type',
       'Overall_Propulsion_Type', 'Specific_Propulsion_Type', 'Areas_Served',
       'Congressional_Districts', 'Address_Line_1', 'City', 'State',
       'Additional_Zip', 'Zip_Code', 'X', 'Y', 'FID2', 'geometry'],
      dtype='object')

(130, 25)

geopandas.geodataframe.GeoDataFrame

Index(['Application_Number', 'FTA_Region', 'Project_Sponsor', 'Project_Title',
       'Project_Description', 'Original_Statement_of_Work', 'Funding',
       'Number_of_Buses', 'Traditional_Buses', 'Low_Emission_Buses',
       'Zero_Emission_Buses', 'Project_Type', 'Display_Type', 'Areas_Served',
       'Congressional_Districts', 'Propulsion_Type', 'Address1', 'Address2',
       'City', 'State', 'Zip', 'X', 'Y', 'ObjectId', 'geometry'],
      dtype='object')